In [1]:
import geopandas as gpd
import pandas as pd
import fiona 
from fiona.errors import DriverError
import os


In [2]:
# Global Vars
OUTPUT_LAYERS_CSV_PATH = r'C:\PERSONAL\UK PHD\NEOM_PROJECT\gdb_layers.csv'
OUTPUT_LAYERS_METADATA_PATH = r'C:\PERSONAL\UK PHD\NEOM_PROJECT\gdb_layer_metadata.csv'
# ROOT_DIRS = [r'C:\PERSONAL\UK PHD\NEOM_PROJECT']
ROOT_DIRS = [r'D:']

In [3]:
# function to return geodatabase / files paths in list 
def get_geodbs_to_list(root_dirs, files_endwith='gdb'):
    """This function walks through directories and grabs all geodatabases"""
    db_names = []
    db_paths = []

    for root_dir in root_dirs:
        for dirpath, dirnames, filenames in os.walk(root_dir):
            for f in dirnames:
                if f.endswith(f"{files_endwith}"):
                    file_path = os.path.join(dirpath, f)
                    
                    # Extract parts 
                    parts = os.path.normpath(file_path).split(os.sep)  
                    name = parts[-2]   

                    db_names.append(name)
                    db_paths.append(file_path)

    return db_names, db_paths


In [4]:
# Get geo dbs to list
gdb_names, gdb_paths = get_geodbs_to_list(root_dirs=ROOT_DIRS)

print(gdb_names)
print(gdb_paths)

['20231217_Final Neom Schema Submission', 'Summer Interim Report GIS', '83474_20240823_Submission', 'Complete dataset for birds and megafauna', '2025-05-11_All Marine Megafauna & Avifauna Geodatabase KMZ', 'FINAL', 'KBD.gdb', 'Avifauna_Artificial_Nest_IR_Rev01', '83474_20240823_Submission', '04_NEOM_FinalData_GDB_20211209', '02_83242_Marine_Species_Recovery_Conservation', '06_83242_MSR_CAP_Dugong', '16_83291_Marine_Megafauna_Offshore_Bird_Survey', '27_83242_MSR_CAP_Dugong_IR_Rev01', '28_83291_NEOM_Marine_Megafauna', '29_83374_NEOM_Marine_Megafauna_Aerial_Survey', '30_83437-NEOM_FDSA_Megafauna', '31_83474_Dugong_M_Megafauna_DVL_Survey', '32_83214_Islands_Turtle_Surveys_Tagging', 'Summer Interim Report GIS', '20231217_Final Neom Schema Submission', '2025-05-11_All Marine Megafauna & Avifauna Geodatabase KMZ', 'FINAL', '5. GIS data', 'Gulf of Aqaba (Magna)', 'Gulf of Aqaba (Magna)', '83393_20251006_Submission', '83393_20251008_Submission', 'Spatial Data', 'Spatial Data', 'Spatial Data', '

In [5]:
# how many geo databases do we have
len(gdb_paths)

48

In [6]:
# find layers in gdb

def get_gdb_layers(gdb_paths, output_layers_csv_path):
    """
    Returns a dictionary mapping each geodatabase path
    to a list of its layer names.
    """
    layers_dict = {}
    rows = []

    for gdb_path in gdb_paths:
        try:
            layers = fiona.listlayers(gdb_path)
            layers_dict[gdb_path] = list(layers)

        except Exception as e:
            print(f"Skipping {gdb_path}: {e}")
            layers_dict[gdb_path] = []  # keep record of failed GDBs
            continue

    # return layers_dict
    for gdb_path, layers in layers_dict.items():
        if layers:
            for layer in layers:
                rows.append({
                    "geodatabase": gdb_path,
                    "layer": layer
                })
        else:
            rows.append({
                "geodatabase": gdb_path,
                "layer": None
            })

        # save a csv
    df = pd.DataFrame(rows)
    df.to_csv(f"{output_layers_csv_path}", index=False)
    
    # return layers
    return rows


In [7]:
# get each geo db and its containing layers
layers = get_gdb_layers(gdb_paths, OUTPUT_LAYERS_CSV_PATH)
layers

Skipping D:Marine New Data\Complete dataset for birds and megafauna\KBD.gdb: Failed to open dataset (flags=68): D:Marine New Data\Complete dataset for birds and megafauna\KBD.gdb


[{'geodatabase': 'D:Marine New Data\\Megafauna (Cetaceans) baseline study\\GIS data\\20231217_Final Neom Schema Submission\\83291_NEOM_MM_OBS_KBD.gdb',
  'layer': 'MM_NEOM_M_V_N_Survey_Boundary'},
 {'geodatabase': 'D:Marine New Data\\Megafauna (Cetaceans) baseline study\\GIS data\\20231217_Final Neom Schema Submission\\83291_NEOM_MM_OBS_KBD.gdb',
  'layer': 'MM_NEOM_M_V_N_Survey_Planned_Transects'},
 {'geodatabase': 'D:Marine New Data\\Megafauna (Cetaceans) baseline study\\GIS data\\20231217_Final Neom Schema Submission\\83291_NEOM_MM_OBS_KBD.gdb',
  'layer': 'MM_NEOM_M_V_N_Winter_Survey_Transects'},
 {'geodatabase': 'D:Marine New Data\\Megafauna (Cetaceans) baseline study\\GIS data\\20231217_Final Neom Schema Submission\\83291_NEOM_MM_OBS_KBD.gdb',
  'layer': 'MM_NEOM_M_V_N_Marine_Megafauna'},
 {'geodatabase': 'D:Marine New Data\\Megafauna (Cetaceans) baseline study\\GIS data\\20231217_Final Neom Schema Submission\\83291_NEOM_MM_OBS_KBD.gdb',
  'layer': 'MM_NEOM_M_V_N_Hydrophone_Deplo

In [8]:
# read the output csv layers table
lyrs_df = pd.read_csv(OUTPUT_LAYERS_CSV_PATH)
lyrs_df.head()

,geodatabase,layer
0,D:Marine New Data\Megafauna (Cetaceans) baseli...,MM_NEOM_M_V_N_Survey_Boundary
1,D:Marine New Data\Megafauna (Cetaceans) baseli...,MM_NEOM_M_V_N_Survey_Planned_Transects
2,D:Marine New Data\Megafauna (Cetaceans) baseli...,MM_NEOM_M_V_N_Winter_Survey_Transects
3,D:Marine New Data\Megafauna (Cetaceans) baseli...,MM_NEOM_M_V_N_Marine_Megafauna
4,D:Marine New Data\Megafauna (Cetaceans) baseli...,MM_NEOM_M_V_N_Hydrophone_Deployment


### READ EACH LAYER AND DERIVE ITS META DATA
- crs
- spatial extent / bbox
- geometry types
- field names
- field data types
- geometry types


In [9]:
# read one database layer to check
gdf = gpd.read_file(lyrs_df.geodatabase[0], layer=lyrs_df.layer[0])
gdf.head()

,globalid,MasterPlan,SurveyCode,SurveyName,SurveyType,AreaName,Author,StartDate,EndDate,Source,...,CreationDate,Creator,EditDate,Editor,SHAPE_Length,SHAPE_Area,Document_Number,DocumentTitle,ReportDate,geometry
0,{EFBFBB28-45A8-44A8-B6A0-9384E0A92E2B},6 Islands & Coastal,M,Marine Megafauna and Offshore Bird Surveys,Fauna,6 (54-73),KAUST Beacon Development,2023-02-20 00:00:00+00:00,2023-03-12 00:00:00+00:00,"KAUST Beacon Development, NEOM - Marine Megafa...",...,2023-12-17 16:56:23+00:00,MOHAMMAY,NaT,MOHAMMAY,404436.092642,7.814854e+08,83291-NEOM-TREP-05DEC23-359-REV02,Marine Megafauna and Offshore Bird Surveys (MM...,10/21/2023,"MULTIPOLYGON (((654657.114 3108506.222, 648984..."
1,{EB69FEEF-6003-4569-A903-9C68802DFC66},6 Islands & Coastal,M,Marine Megafauna and Offshore Bird Surveys,Fauna,1 (1-18),KAUST Beacon Development,2023-02-20 00:00:00+00:00,2023-03-12 00:00:00+00:00,"KAUST Beacon Development, NEOM - Marine Megafa...",...,2023-12-17 16:56:23+00:00,MOHAMMAY,NaT,MOHAMMAY,147036.492603,4.382860e+08,83291-NEOM-TREP-05DEC23-359-REV02,Marine Megafauna and Offshore Bird Surveys (MM...,10/21/2023,"MULTIPOLYGON (((686683.735 3098148.815, 686683..."
2,{E27D9835-4C30-4706-83EF-061196026FAC},6 Islands & Coastal,M,Marine Megafauna and Offshore Bird Surveys,Fauna,2 (19-30),KAUST Beacon Development,2023-02-20 00:00:00+00:00,2023-03-12 00:00:00+00:00,"KAUST Beacon Development, NEOM - Marine Megafa...",...,2023-12-17 16:56:23+00:00,MOHAMMAY,NaT,MOHAMMAY,189127.365156,2.744758e+08,83291-NEOM-TREP-05DEC23-359-REV02,Marine Megafauna and Offshore Bird Surveys (MM...,10/21/2023,"MULTIPOLYGON (((678413.607 3096181.335, 660311..."
3,{CD698CA6-ABF4-4D72-B7AF-C4FEAC66F3E8},6 Islands & Coastal,M,Marine Megafauna and Offshore Bird Surveys,Fauna,3 (31-39),KAUST Beacon Development,2023-02-20 00:00:00+00:00,2023-03-12 00:00:00+00:00,"KAUST Beacon Development, NEOM - Marine Megafa...",...,2023-12-17 16:56:23+00:00,MOHAMMAY,NaT,MOHAMMAY,107121.173935,3.754403e+08,83291-NEOM-TREP-05DEC23-359-REV02,Marine Megafauna and Offshore Bird Surveys (MM...,10/21/2023,"MULTIPOLYGON (((711103.224 3081132.711, 711062..."
4,{8E3B8461-B8D9-4909-B61A-77549607B1B0},6 Islands & Coastal,M,Marine Megafauna and Offshore Bird Surveys,Fauna,5 (50-53),KAUST Beacon Development,2023-02-20 00:00:00+00:00,2023-03-12 00:00:00+00:00,"KAUST Beacon Development, NEOM - Marine Megafa...",...,2023-12-17 16:56:23+00:00,MOHAMMAY,NaT,MOHAMMAY,96039.941443,4.792009e+08,83291-NEOM-TREP-05DEC23-359-REV02,Marine Megafauna and Offshore Bird Surveys (MM...,10/21/2023,"MULTIPOLYGON (((711103.224 3081132.711, 699805..."


Extract meta data for all layers

In [17]:
# function to extract layers meta data

def extract_gdb_layer_metadata(
    layers_df,
    output_csv
):
    """
    Reads geodatabase layers and extracts metadata safely.
    
    Parameters
    ----------
    layers_df : pd.DataFrame
        Must contain columns: ['geodatabase', 'layer']
    output_csv : str
        Path to save metadata CSV
    
    Returns
    -------
    pd.DataFrame
        Detailed metadata table
    """

    records = []

    for idx, row in layers_df.iterrows():
        gdb = row["geodatabase"]
        layer = row["layer"]

        meta = {
            "geodatabase": gdb,
            "layer": layer,
            "status": "success",
            "error": None
        }
   

        try:
            # ---- Read layer ----
            gdf = gpd.read_file(gdb, layer=layer)

            # ---- Spatial metadata ----
            meta["crs"] = str(gdf.crs)
            meta["epsg"] = gdf.crs.to_epsg() if gdf.crs else None
            meta["geometry_types"] = ", ".join(sorted(gdf.geom_type.unique()))
            meta["bbox"] = list(gdf.total_bounds)

            # ---- Feature-level metadata ----
            meta["feature_count"] = len(gdf)
            meta["has_geometry"] = "geometry" in gdf.columns

            # ---- Attribute metadata ----
            meta["field_count"] = len(gdf.columns)
            meta["field_names"] = ", ".join(gdf.columns)

            meta["field_types"] = ", ".join(
                f"{col}:{dtype}"
                for col, dtype in gdf.dtypes.items()
            )

            parts = layer.split('_')
            meta["first_word"] = f"{parts[0]}_{parts[1]}"

            # ---- Derived metadata ----
            meta["memory_mb"] = round(
                gdf.memory_usage(deep=True).sum() / (1024 ** 2), 3
            )

            # ---- Z / M detection (best-effort) ----
            try:
                meta["has_z"] = gdf.geometry.has_z.any()
            except Exception:
                meta["has_z"] = None

        except (DriverError, PermissionError) as e:
            meta["status"] = "skipped"
            meta["error"] = str(e)

        except Exception as e:
            meta["status"] = "failed"
            meta["error"] = str(e)

        records.append(meta)

    # ---- Create DataFrame ----
    meta_df = pd.DataFrame(records)

    # ---- Save CSV ----
    meta_df.to_csv(output_csv, index=False, encoding="utf-8")

    return meta_df



In [18]:
# extract meta data for each geo database layer and save csv
metadata_df = extract_gdb_layer_metadata(
    lyrs_df,
    output_csv=OUTPUT_LAYERS_METADATA_PATH
)

metadata_df.head()  

c:\PERSONAL\UK PHD\NEOM_PROJECT\neom\Lib\site-packages\pyogrio\raw.py:200: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D MultiLineString' is converted to 'MultiLineString Z'
  return ogr_read(
c:\PERSONAL\UK PHD\NEOM_PROJECT\neom\Lib\site-packages\pyogrio\raw.py:200: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D MultiLineString' is converted to 'MultiLineString Z'
  return ogr_read(
c:\PERSONAL\UK PHD\NEOM_PROJECT\neom\Lib\site-packages\pyogrio\raw.py:200: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D MultiLineString' is converted to 'MultiLineString Z'
  return ogr_read(
c:\PERSONAL\UK PHD\NEOM_PROJECT\neom\Lib\site-packages\pyogrio\raw.py:200: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D MultiLineString' is converted to 'MultiLineString Z'
  return ogr_read(
c:\PERSONAL\UK PHD\NEOM_PROJECT\neom\Lib\site-packages\pyogrio\r

,geodatabase,layer,status,error,crs,epsg,geometry_types,bbox,feature_count,has_geometry,field_count,field_names,field_types,first_word,memory_mb,has_z
0,D:Marine New Data\Megafauna (Cetaceans) baseli...,MM_NEOM_M_V_N_Survey_Boundary,success,None,EPSG:32636,32636.0,MultiPolygon,"[643259.7836999996, 3010869.1075, 764940.87880...",8.0,True,23.0,"globalid, MasterPlan, SurveyCode, SurveyName, ...","globalid:object, MasterPlan:object, SurveyCode...",MM_NEOM,0.011,False
1,D:Marine New Data\Megafauna (Cetaceans) baseli...,MM_NEOM_M_V_N_Survey_Planned_Transects,success,None,EPSG:4326,4326.0,MultiLineString,"[34.45928255100006, 27.268224460000056, 35.648...",97.0,True,30.0,"GlobalID, MasterPlan, SurveyCode, SurveyName, ...","GlobalID:object, MasterPlan:object, SurveyCode...",MM_NEOM,0.158,False
2,D:Marine New Data\Megafauna (Cetaceans) baseli...,MM_NEOM_M_V_N_Winter_Survey_Transects,success,None,EPSG:4326,4326.0,MultiLineString,"[34.53615328300003, 27.269729879000067, 35.695...",19.0,True,30.0,"GlobalID, MasterPlan, SurveyCode, SurveyName, ...","GlobalID:object, MasterPlan:object, SurveyCode...",MM_NEOM,0.032,False
3,D:Marine New Data\Megafauna (Cetaceans) baseli...,MM_NEOM_M_V_N_Marine_Megafauna,success,None,EPSG:4326,4326.0,Point,"[34.535942000000034, 27.397659000000033, 35.45...",65.0,True,43.0,"globalid, MasterPlan, SurveyCode, SurveyName, ...","globalid:object, MasterPlan:object, SurveyCode...",MM_NEOM,0.143,False
4,D:Marine New Data\Megafauna (Cetaceans) baseli...,MM_NEOM_M_V_N_Hydrophone_Deployment,success,None,EPSG:4326,4326.0,Point,"[34.51054400000004, 27.26828100000006, 35.6478...",157.0,True,30.0,"GlobalID, MasterPlan, SurveyCode, SurveyName, ...","GlobalID:object, MasterPlan:object, SurveyCode...",MM_NEOM,0.274,False


Questions
- What is the meaning of habitat groups, core groups and core names
- What is the difference between core groups, core names and habitat groups

Read Layer 2

### Get all images meta data table

In [12]:
# function to grab all images in the directory 
images_extenstions = ['png', 'jpg']

def get_images_to_list(root_dirs, files_endwith='png'):
    """This function walks through directories and grabs all geodatabases"""
    db_names = []
    db_paths = []

    for root_dir in root_dirs:
        for dirpath, dirnames, filenames in os.walk(root_dir):
            for f in filenames:
                if f.endswith(f"{files_endwith}"):
                    file_path = os.path.join(dirpath, f)
                    
                    # Extract parts 
                    # parts = os.path.normpath(file_path).split(os.sep)  
                    # name = parts[-2]   

                    # db_names.append(name)
                    db_paths.append(file_path)

    return db_paths

In [13]:


images = get_images_to_list(ROOT_DIRS)985
images

['D:Marine New Data\\Megafauna (Cetaceans) baseline study\\GIS data\\NEOM Marine Megafauna Density Maps\\NEOM Cetacean Sighting Density Map.png',
 'D:Marine New Data\\Megafauna (Cetaceans) baseline study\\GIS data\\NEOM Marine Megafauna Density Maps\\._NEOM Cetacean Sighting Density Map.png',
 'D:Marine New Data\\Megafauna (Cetaceans) baseline study\\GIS data\\NEOM Marine Megafauna Density Maps\\NEOM Turtle Sighting Density Map.png',
 'D:Marine New Data\\Megafauna (Cetaceans) baseline study\\GIS data\\NEOM Marine Megafauna Density Maps\\._NEOM Turtle Sighting Density Map.png',
 'D:Marine New Data\\Megafauna (Cetaceans) baseline study\\GIS data\\NEOM Marine Megafauna Density Maps\\NEOM Dugong Sighting Density Map.png',
 'D:Marine New Data\\Megafauna (Cetaceans) baseline study\\GIS data\\NEOM Marine Megafauna Density Maps\\._NEOM Dugong Sighting Density Map.png',
 'D:Marine New Data\\Megafauna (Cetaceans) baseline study\\05_Winter Interim Report\\Interim Report Photographs\\GX014225_Sten

In [14]:
len(images)

214